#  Extracting data on Sold properties with Domain api

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Roudra Das    roudra.das@gmail.com                                            |
| External References   | <a href="https://api.domain.com.au/" target="_blank">Domain API</a>|
| Input Datasets        |  List For Sale |
| Output Datasets       |  Table|
| Input Data Source     |  API |
| Output Data Source    | Pandas Dataframe |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 15th May 2021 | Roudra Das | Notebook created to estimate price range of properties on the market in Australia|

## Other Details
This Notebook is a prototype.

In [2]:
import json
import requests # this library is awesome: http://docs.python-requests.org/en/master/
import re, string, timeit
import time
import pandas as pd

In [3]:
# setup
property_id="2016836733"
starting_max_price=100000
increment=20000
# when starting min price is zero we'll just use the lower bound plus 400k later on
starting_min_price=0

In [ ]:
## Functions

In [4]:
def get_api_key(api_key_id = "Domain"):
  """
  Get the api key for website accessing.

  Table of key type and key value for privacy.

  Parameters
  ----------
  @api_key_id [string]: Key value in dataframe

  Returns
  -------
  [string]: client_id & client_secret

  """
  # load api keys file
  df_api_keys = pd.read_csv('~/Documents/Python/api_keys.csv', header = 'infer')
  
  # return api key if in dataset
  try:
    # get api key from id
    client_id = df_api_keys.loc[df_api_keys['Id'] == api_key_id]['Client'].iloc[0] # get client by id
    client_secret = df_api_keys.loc[df_api_keys['Id'] == api_key_id]['Secret'].iloc[0] # get secret by id
    # return api key
    return client_id, client_secret
  except IndexError:
    # get api key id list
    api_key_id_list = df_api_keys['Id'].unique().tolist()
    # print error message
    print('Cannot map key. Api key id must be one of the following options {0}'.format(api_key_id_list))

In [5]:
# POST request for token
client_id, client_secret = get_api_key(api_key_id="Domain")
response = requests.post('https://auth.domain.com.au/v1/connect/token', data = {'client_id':client_id,"client_secret":client_secret,"grant_type":"client_credentials","scope":"api_listings_read","Content-Type":"text/json"})
token=response.json()
access_token=token["access_token"]

In [6]:
# GET Request for ID
url = "https://api.domain.com.au/v1/listings/"+property_id
auth = {"Authorization":"Bearer "+access_token}
request = requests.get(url,headers=auth)
r=request.json()

In [7]:
def json_to_dataframe(response):
    """
    Convert response to dataframe
    """
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [8]:
request.json()

{'objective': 'sale',
 'propertyTypes': ['house'],
 'status': 'sold',
 'saleMode': 'sold',
 'channel': 'residential',
 'addressParts': {'stateAbbreviation': 'nsw',
  'displayType': 'fullAddress',
  'streetNumber': '9',
  'street': 'Worsley Place',
  'suburb': 'Lavington',
  'postcode': '2641',
  'displayAddress': '9 Worsley Place, Lavington NSW 2641'},
 'advertiserIdentifiers': {'advertiserType': 'agency',
  'advertiserId': 11631,
  'contactIds': [1193890, 1400351],
  'agentIds': ['A4678', 'A4624']},
 'bathrooms': 2.0,
 'bedrooms': 4.0,
 'carspaces': 3.0,
 'dateUpdated': '2021-04-06T07:13:02.843Z',
 'dateListed': '2021-03-02T05:49:48Z',
 'description': 'An exceptional floor plan with space for the family, 9 Worsley Place is also conveniently located in a quiet court, just steps to local schools and easy access to the Lavington Shopping complex.\n\nOffering two living areas, the formal lounge and dining area are positioned at the front of the home. The family and kitchen area open to th

In [9]:
#get details
da=r['addressParts']
postcode=da['postcode']
suburb=da['suburb']
bathrooms=r['bathrooms']
bedrooms=r['bedrooms']
carspaces=r['carspaces']
property_type=r['propertyTypes']
area=r['landAreaSqm']
geolocation=r['geoLocation']
print(property_type,postcode, suburb, bedrooms, bathrooms,  carspaces, area, geolocation)

# the below puts all relevant property types into a single string. eg. a property listing can be a 'house' and a 'townhouse'
n=0
property_type_str=""
for p in r['propertyTypes']:
  property_type_str=property_type_str+(r['propertyTypes'][int(n)])
  n=n+1
print(property_type_str) 

['house'] 2641 Lavington 4.0 2.0 3.0 720.0 {'latitude': -36.0337125, 'longitude': 146.9383104}
house


In [10]:
max_price=starting_max_price
searching_for_price=True

In [11]:
# Start your loop
while searching_for_price:
    
    url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here
    post_fields ={
      "listingType":"Sale",
        "maxPrice":max_price,
        "pageSize":100,
      "propertyTypes":property_type,
      "minBedrooms":bedrooms,
        "maxBedrooms":bedrooms,
      "minBathrooms":bathrooms,
        "maxBathrooms":bathrooms,
      "locations":[
        {
          "state":"",
          "region":"",
          "area":"",
          "suburb":suburb,
          "postCode":postcode,
          "includeSurroundingSuburbs":False
        }
      ]
    }

    request = requests.post(url,headers=auth,json=post_fields)

    l=request.json()
    listings = []
    for listing in l:
        listings.append(listing["listing"]["id"])
    listings

    if int(property_id) in listings:
            max_price=max_price-increment
            print("Lower bound found: ", max_price)
            searching_for_price=False
    else:
        max_price=max_price+increment
        print("Not found. Increasing max price to ",max_price)
        time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly 

Not found. Increasing max price to  120000
Not found. Increasing max price to  140000
Not found. Increasing max price to  160000
Not found. Increasing max price to  180000
Not found. Increasing max price to  200000
Not found. Increasing max price to  220000
Not found. Increasing max price to  240000
Not found. Increasing max price to  260000
Not found. Increasing max price to  280000
Not found. Increasing max price to  300000
Not found. Increasing max price to  320000
Not found. Increasing max price to  340000
Not found. Increasing max price to  360000
Not found. Increasing max price to  380000
Not found. Increasing max price to  400000
Not found. Increasing max price to  420000
Not found. Increasing max price to  440000
Not found. Increasing max price to  460000
Not found. Increasing max price to  480000
Not found. Increasing max price to  500000
Not found. Increasing max price to  520000
Not found. Increasing max price to  540000
Not found. Increasing max price to  560000
Not found. 

KeyboardInterrupt: 

In [42]:
searching_for_price=True
if starting_min_price>0:
  min_price=starting_min_price
else:  
  min_price=max_price+400000 

In [43]:
while searching_for_price:
    
    url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here
    post_fields ={
      "listingType":"Sale",
        "minPrice":min_price,
        "pageSize":100,
      "propertyTypes":property_type,
      "minBedrooms":bedrooms,
        "maxBedrooms":bedrooms,
      "minBathrooms":bathrooms,
        "maxBathrooms":bathrooms,
      "locations":[
        {
          "state":"",
          "region":"",
          "area":"",
          "suburb":suburb,
          "postCode":postcode,
          "includeSurroundingSuburbs":False
        }
      ]
    }

    request = requests.post(url,headers=auth,json=post_fields)

    l=request.json()
    listings = []
    for listing in l:
        listings.append(listing["listing"]["id"])
    listings

    if int(property_id) in listings:
            min_price=min_price+increment
            print("Upper bound found: ", min_price)
            searching_for_price=False
    else:
        min_price=min_price-increment
        print("Not found. Decreasing min price to ",min_price)
        time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly 

Not found. Decreasing min price to  720000
Not found. Decreasing min price to  700000
Not found. Decreasing min price to  680000
Not found. Decreasing min price to  660000
Not found. Decreasing min price to  640000
Not found. Decreasing min price to  620000
Not found. Decreasing min price to  600000
Not found. Decreasing min price to  580000
Not found. Decreasing min price to  560000
Not found. Decreasing min price to  540000
Not found. Decreasing min price to  520000
Not found. Decreasing min price to  500000
Not found. Decreasing min price to  480000
Not found. Decreasing min price to  460000
Not found. Decreasing min price to  440000
Not found. Decreasing min price to  420000
Not found. Decreasing min price to  400000
Not found. Decreasing min price to  380000
Upper bound found:  400000


In [44]:
if max_price<1000000:
  lower=max_price/1000
  upper=min_price/1000
  denom="k"
else: 
  lower=max_price/1000000
  upper=min_price/1000000
  denom="m"

In [45]:
# Print the results
print(da['displayAddress'])
print(r['headline'])
print("Property Type:",property_type_str)
print("Details: ",int(bedrooms),"bedroom,",int(bathrooms),"bathroom,",int(carspaces),"carspace", int(area),"sqm")
print("Geocode: ", geolocation)
print("Display price:",r['priceDetails']['displayPrice'])      
if max_price==min_price:
  print("Price guide:","$",lower,denom)
else:
  print("Price range:","$",lower,"-","$",upper,denom)
print("URL:",r['seoUrl'])

9 Worsley Place, Lavington NSW 2641
Family Living In A Quiet Court
Property Type: house
Details:  4 bedroom, 2 bathroom, 3 carspace 720 sqm
Geocode:  {'latitude': -36.0337125, 'longitude': 146.9383104}
Display price: Contact Agent
Price range: $ 340.0 - $ 400.0 k
URL: https://www.domain.com.au/9-worsley-place-lavington-nsw-2641-2016836733


In [46]:
min_price = 0

In [47]:
def search_for_price(property_type, bedrooms, bathrooms, suburb, postcode, starting_min_price, starting_max_price, increment):
    
    url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here

    max_price=starting_max_price

    searching_for_price_l = True
    while searching_for_price_l:
        post_fields ={
        "listingType":"Sale",
            "maxPrice":max_price,
            "pageSize":100,
        "propertyTypes":property_type,
        "minBedrooms":bedrooms,
            "maxBedrooms":bedrooms,
        "minBathrooms":bathrooms,
            "maxBathrooms":bathrooms,
        "locations":[
            {
            "state":"",
            "region":"",
            "area":"",
            "suburb":suburb,
            "postCode":postcode,
            "includeSurroundingSuburbs":False
            }
        ]
        }

        request = requests.post(url,headers=auth,json=post_fields)

        l=request.json()
        listings = []
        for listing in l:
            listings.append(listing["listing"]["id"])
        listings

        if int(property_id) in listings:
                max_price=max_price-increment
                print("Lower bound found: ", max_price)
                searching_for_price_l=False
        else:
            max_price=max_price+increment
            print("Not found. Decreasing min price to ",max_price)
            time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly )


    if starting_min_price>0:
            min_price=starting_min_price
            
    else:  
            min_price=max_price+400000


    searching_for_price_u = True
    while searching_for_price_u:
        post_fields ={
        "listingType":"Sale",
            "minPrice":min_price,
            "pageSize":100,
        "propertyTypes":property_type,
        "minBedrooms":bedrooms,
            "maxBedrooms":bedrooms,
        "minBathrooms":bathrooms,
            "maxBathrooms":bathrooms,
        "locations":[
            {
            "state":"",
            "region":"",
            "area":"",
            "suburb":suburb,
            "postCode":postcode,
            "includeSurroundingSuburbs":False
            }
        ]
        }

        request = requests.post(url,headers=auth,json=post_fields)

        l=request.json()
        listings = []
        for listing in l:
            listings.append(listing["listing"]["id"])
        listings

        if int(property_id) in listings:
                min_price=min_price+increment
                print("Upper bound found: ", min_price)
                searching_for_price_u=False
        else:
            min_price=min_price-increment
            print("Not found. Decreasing min price to ",min_price)
            time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly )

        if max_price<1000000:
            lower=max_price/1000
            upper=min_price/1000
            denom="k"
        else: 
            lower=max_price/1000000
            upper=min_price/1000000
            denom="m"
    

    return print("Price range:","$",lower,"-","$",upper,denom)

In [48]:
#search_for_price(property_type, bedrooms, bathrooms, suburb, postcode, 0, 100000, 50000)

# End of Notebook